# Obtenir dataset películes

En aquest codi anem a obtenir un dataset de pel·lícules.
Realitzarem els següents pasos:
- Cargar llibreries.
- Realitzar web scraping sobre la web https://www.themoviedb.org/movie/
- Capturar el llistat de totes les pelicules i obtenir els enllaços
- Accedir a cada pàgina de la pelìcula i agafar les dades
- Tractar aquestes dades i guardar-les en un dataset.
- Visualitzar el dataset per veure si s'ha creat correctament.

### Cargar llibreries

In [338]:
from selenium.webdriver.chrome.options import Options 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

### Iniciar consulta a la web

In [359]:
url_base = "https://www.themoviedb.org"

options = Options()
# Afegirem l'opció --headless per a que s'execute en segon planol i no obriga el navegador
options.add_argument("--headless")
# Buscarem les pelicules en la versio en espanyol de la web
options.add_argument("--lang=es-ES");

driver = webdriver.Chrome("/usr/local/bin/chromedriver", options=options)
driver.get(url_base+"/movie")

In [360]:
# Obrir les pestanyes a l'esquerra per tal de fer la selecció dels filtres
for item in driver.find_elements_by_xpath(".//*[@class='filter_panel card closed']"):
    ActionChains(driver).move_to_element(item).click().perform()
time.sleep(1)

# Seleccionar pel·lícules d'acció i de Netflix
accio = driver.find_element_by_link_text("Acción")
driver.execute_script("arguments[0].click();", accio)

Netflix = driver.find_element_by_xpath("//a[@title='Netflix']")
driver.execute_script("arguments[0].click();", Netflix)

time.sleep(0.5)

# Fer click al botó de Buscar que apareix
buscar = driver.find_element_by_link_text("Buscar")
driver.execute_script("arguments[0].click();", buscar)
time.sleep(1)

### Obtindre les pelicules del llistat.

Utilitzarem Selenium per a interactuar amb la web

Primer farem clic en mostrar más i desprès anarim carregant les dades mitjançant l'scroll.

Definim un maxim de vegades que farem scroll

In [182]:
# Fer click en "Mostrar más"
element = driver.find_element_by_link_text("Mostrar más") 
driver.execute_script("arguments[0].click();", element)
time.sleep(1)

# Fer scroll per capturar dades conforme es van carregant
url_single_movies = []
max_iterations = 0
iter=1
while max_iterations < 1:
    scroll_height = driver.execute_script("return document.documentElement.scrollHeight")
    # A cada iteració anem augmentant l'scroll
    height=4200*iter
    driver.execute_script("window.scrollTo(0, " + str(height) + ");")
    # Si hem arribat al final de la pàgina, parem el bucle
    if height > scroll_height:
        print('End of page')
        break
    time.sleep(random.randint(1, 3))
    max_iterations+=1
    iter+=1

body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML') 

# Agafem les pelicules
soup = BeautifulSoup(source, "html.parser")
movies = soup.find_all("div", {"class": "card style_1"})

print(len(movies))

40


### Obtenir informació individual de cada pel·lícula

Primer recorrerem l'HTML obtingut i capturarem els enllaços a la fitxa de cada pel·lícula

In [183]:
# Capturar els enllaços
for movie in movies:
    title = movie.h2.a["href"]
    url_single_movies.append(title)

Creem algunes funcions auxiliars que ens ajudaran a tractar les dades de la fitxa. A mes, si en un futur l'estructura de la pagina que estem fent web scraping canvia, podem modificar mes facilment la part que corresponga.

Si no existeix algún valor retornarem "?" en tots els camps excepte amb "Puntuación" que retornarem un -1.

In [184]:
# Get Título
def get_titulo(movie):
    try:
        return movie.find("h2").a.contents[0]
    except AttributeError:
        return "?"

# Get Título original
def get_titulo_otiginal(movie):
    try:
        titulo_container = movie.find("p", {"class": "wrap"}).get_text()
        return titulo_container.split("Título original", 1)[1]
    except AttributeError:
        return "?"

# Get Idioma original
def get_idioma_original(movie):
    try:
        idioma_container = movie.get_text().split("Idioma original", 1)
        return idioma_container[1].split("\n", 1)[0]
    except AttributeError:
        return "?"

# Get Duración
def get_duracion(movie):
    try:
        return movie.find("span", {"class":"runtime"}).text.strip()
    except AttributeError:
        return "?"

# Get Géneros
def get_generos(movie):
    try:
        generos_container = movie.find("span", {"class":"genres"}).find_all("a")
        generos_container = [generos_container[genre].text for genre in range(len(generos_container))]
        return ", ".join(generos_container)
    except AttributeError:
        return "?"
    
# Get Directores
def get_directores(movie):
    try:
        directores = []
        people_container = movie.select("ol.people.no_image li")
        for people in people_container:
            cargo = people.find("p", {"class":"character"}).text
            if "Director" in cargo:
                directores.append(people.find("p").text)
        return ", ".join(directores)
    except AttributeError:
        return "?"

# Get Sinopsis
def get_sinopsis(movie):
    try:
        return movie.find_all("div",{"class": "overview"})[0].text.strip()
    except AttributeError:
        return "?"

# Get Fecha de estreno
def get_fecha_estreno(movie):
    try:
        return movie.find("span", {"class": "release"}).get_text().strip().split(" ", 1)[0]
    except AttributeError:
        return "?"

# Get Clasificación por edad
def get_clasificacion_edad(movie):
    try:
        return movie.find("span", {"class": "certification"}).get_text().strip()
    except AttributeError:
        return "?"

# Get Imagen cartel
def get_img_cartel(movie):
    try:
        return movie.find("img").attrs['src']
    except AttributeError:
        return "?"

# Get Puntiación
def get_puntuacion(movie):
    try:
        return list(movie.find("span",{"class":"icon"}).attrs.values())[0][1][-2:]
    except AttributeError:
        return -1


Obrim el fitxer csv on guardarem les dades obtingudes i escrivim la primera línea que sera el nom de les columnes

In [185]:
data_file = open('dataset_peliculas.csv', 'w', newline='', encoding='utf-8') 
csv_writer = csv.writer(data_file, delimiter=";") 
csv_writer.writerow(["Título","Título original","Idioma original","Duración","Géneros","Directores","Sinopsis","Fecha de estreno","Clasificación por edad","Imagen cartel","Puntuación"]) 

142

Accedim a la fitxa de cada pel·lícula i anem capturant les dades i guardant-les en el fitxer csv

In [186]:
# Recorrem els enllaços obtinguts i anem obtinguent informació de cada pel·lícula
for url_movie in url_single_movies:
    driver.get(url_base+url_movie)
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML') 
    soup = BeautifulSoup(source, "html.parser")

    # Contenidors de la informació que volem capturar
    poster_wrapper = soup.find("div", {"class": "poster_wrapper"})
    header_wrapper = soup.find("div", {"class": "header_poster_wrapper"})
    column_wrapper = soup.find("section", {"class": "left_column"})

    movie =  [
        get_titulo(header_wrapper), # Título
        get_titulo_otiginal(column_wrapper), # Título original
        get_idioma_original(column_wrapper), # Idioma original
        get_duracion(header_wrapper), # Duración
        get_generos(header_wrapper), # Géneros        
        get_directores(header_wrapper), # Directores
        get_sinopsis(header_wrapper), # Sinopsis
        get_fecha_estreno(header_wrapper), # Fecha de estreno
        get_clasificacion_edad(header_wrapper), # Clasificación por edad
        url_base+get_img_cartel(poster_wrapper), # Imagen cartel
        get_puntuacion(header_wrapper) # Puntuación 
    ]

    csv_writer.writerow(movie)
    
    # Deixem entre 1 i 3 segunds de pausa entre cada iteració per no saturar la web
    time.sleep(random.randint(1, 2))

Tanquem el fitxer csv

In [187]:
data_file.close()

Tanquem el driver de selenium

In [188]:
driver.close()

### Comprovació final

Per comprovar que s'han guardat correctament, carreguem el csv en una variable i vegem els primers registres

In [285]:
df_movies = pd.read_csv("dataset_peliculas.csv", sep=";", encoding="utf-8")
df_movies.head(10)

,Título,Título original,Idioma original,Duración,Géneros,Directores,Sinopsis,Fecha de estreno,Clasificación por edad,Imagen cartel,Puntuación
0,Raya y el último dragón,Raya and the Last Dragon,Inglés,1h 30m,"Animación, Aventura, Fantasía, Familia, Acción","Carlos López Estrada, Don Hall","Raya, una niña de gran espíritu aventurero, s...",5/3/2021,APTA,https://www.themoviedb.org/t/p/w300_and_h450_b...,85
1,Centinela,Sentinelle,Francés,1h 20m,"Suspense, Acción, Drama",Julien Leclercq,Enviada a casa tras una traumática misión de c...,5/3/2021,NR,https://www.themoviedb.org/t/p/w300_and_h450_b...,61
2,Bajocero,?,Español; Castellano,1h 41m,"Acción, Crimen, Suspense",Lluís Quílez,Invierno. Bajo cero. Noche cerrada. En mitad d...,29/1/2021,16,https://www.themoviedb.org/t/p/w300_and_h450_b...,64
3,Tom y Jerry,Tom & Jerry,Inglés,1h 41m,"Acción, Comedia, Familia, Animación",Tim Story,Tom el gato y Jerry el ratón son expulsados de...,26/2/2021,PG,https://www.themoviedb.org/t/p/w300_and_h450_b...,75
4,Monster Hunter,?,Inglés,1h 44m,"Fantasía, Acción, Aventura",Paul W. S. Anderson,Tras nuestro mundo hay otro: un mundo de monst...,26/3/2021,PG-13,https://www.themoviedb.org/t/p/w300_and_h450_b...,72
5,Wonder Woman 1984,?,Inglés,2h 31m,"Fantasía, Acción, Aventura",Patty Jenkins,"En 1984, en plena guerra fría, Diana Prince, c...",18/12/2020,12,https://www.themoviedb.org/t/p/w300_and_h450_b...,68
6,La Liga de la Justicia de Zack Snyder,Zack Snyder's Justice League,Inglés,4m,"Acción, Aventura, Ciencia ficción, Fantasía",Zack Snyder,Decidido a garantizar que el último sacrificio...,18/3/2021,18,https://www.themoviedb.org/t/p/w300_and_h450_b...,89
7,El rey de Zamunda,Coming 2 America,Inglés,1h 50m,Comedia,Craig Brewer,¡Akeem y Senmi han vuelto! Desde el exuberante...,5/3/2021,12,https://www.themoviedb.org/t/p/w300_and_h450_b...,70
8,Barrenderos espaciales,승리호,Coreano,2h 16m,"Drama, Fantasía, Ciencia ficción",Jo Sung-hee,"En 2092, la tripulación de una nave basurero e...",5/2/2021,PG-13,https://www.themoviedb.org/t/p/w300_and_h450_b...,73
9,Abismo,Black Water: Abyss,Inglés,1h 38m,"Terror, Suspense, Aventura, Misterio",Andrew Traucki,La película sigue a una pareja amante de la av...,7/8/2020,NR,https://www.themoviedb.org/t/p/w300_and_h450_b...,50
